In [77]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [78]:
device = "cuda" if torch.cuda.is_available() else "cpu"

in_channel = 3
num_classes = 10
learning_rate = 0.001
batch_size = 1024
EPOCHS = 2

In [79]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [80]:
model = torchvision.models.vgg16(pretrained=True)
model.avgpool = Identity()
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [81]:
train_dataset = datasets.CIFAR10(root = "../../data/CIFAR", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [82]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
for epoch in range(EPOCHS):
    for batch_idx, (data, label) in enumerate(train_loader):

        data = data.to(device=device)
        label = label.to(device=device)

        y_pred = model(data)
        loss = loss_fn(y_pred, label)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    
        if batch_idx % 10 == 0:
            print(loss.item())

def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")


2.3406944274902344
1.89801824092865
1.6471158266067505
1.4289391040802002
1.2856855392456055
1.2490335702896118
1.2633335590362549
1.240338921546936
1.179491400718689
1.2540032863616943
Accuracy on training set: 59.25
